In [83]:
import polars as pl
import numpy as np 
import matplotlib.pyplot as plt 
import math

In [174]:
betsize:float = 10000.0

trade_hist = [
    {"timestamp":1, "position":0.5, "price":60000.0},
    {"timestamp":2, "position":1, "price":65000.0},
    {"timestamp":3, "position":1, "price":70000.0},
    {"timestamp":4, "position":0, "price":71000},
     {"timestamp":5, "position":0, "price":80000},
]

test_df = pl.DataFrame(trade_hist)
print(test_df)

shape: (5, 3)
┌───────────┬──────────┬─────────┐
│ timestamp ┆ position ┆ price   │
│ ---       ┆ ---      ┆ ---     │
│ i64       ┆ f64      ┆ f64     │
╞═══════════╪══════════╪═════════╡
│ 1         ┆ 0.5      ┆ 60000.0 │
│ 2         ┆ 1.0      ┆ 65000.0 │
│ 3         ┆ 1.0      ┆ 70000.0 │
│ 4         ┆ 0.0      ┆ 71000.0 │
│ 5         ┆ 0.0      ┆ 80000.0 │
└───────────┴──────────┴─────────┘


# Fix Bet size Position 問題

Previously, I define the PnL as 

$$
PnL = \text{cumQty} * \text{price} - \text{lastPosition}*\text{betSize} - \text{transCost}
$$

This defintion is looking at the difference between last position (in USDT) and the current cumulative holding (in USDT) with the transaction cost at particular moment. This defintion cannot capture the price changes of the asset? 

```
┌───────────┬──────────┬───────┬───────────────┬───┬────────────┬─────────┬─────────┬─────────────┐
│ timestamp ┆ position ┆ price ┆ target_amount ┆ … ┆ trans_cost ┆ PnL     ┆ cum_PnL ┆ cum_holding │
│ ---       ┆ ---      ┆ ---   ┆ ---           ┆   ┆ ---        ┆ ---     ┆ ---     ┆ ---         │
│ i64       ┆ f64      ┆ i64   ┆ f64           ┆   ┆ f64        ┆ f64     ┆ f64     ┆ f64         │
╞═══════════╪══════════╪═══════╪═══════════════╪═══╪════════════╪═════════╪═════════╪═════════════╡
│ 1         ┆ 0.5      ┆ 60000 ┆ 5000.0        ┆ … ┆ 2.988      ┆ -2.988  ┆ -2.988  ┆ 9997.012    │
│ 2         ┆ 1.0      ┆ 65000 ┆ 10000.0       ┆ … ┆ 2.73       ┆ 392.27  ┆ 389.282 ┆ 10389.282   │
│ 3         ┆ 1.0      ┆ 70000 ┆ 10000.0       ┆ … ┆ 0.462      ┆ 709.538 ┆ 1098.82 ┆ 11098.82    │
│ 4         ┆ 1.0      ┆ 70050 ┆ 10000.0       ┆ … ┆ 0.0        ┆ -52.9   ┆ 1045.92 ┆ 11045.92    │
└───────────┴──────────┴───────┴───────────────┴───┴────────────┴─────────┴─────────┴─────────────┘
```

Look at the above table, at time 4 the price of the asset increases to 70050 but the PnL is negative. The reason is that the PnL I refer as the price deviation from target amount and cumulative holding in USDT. Therfore, 70050 * 0.142 =9,947.1 < 10000, resulting in neagtive PnL.


# Update 

$$
PnL = \text{cumQty} * (\text{currPrice} - \text{lastPrice}) - \text{transCost}
$$


In [175]:
test_df = test_df.with_columns(
    (
        pl.col("price").pct_change().fill_null(0)
    ).alias("price_chg")
)

test_df = test_df.with_columns(
    (pl.col("position") * betsize).alias("target_amount")
)



test_df = test_df.with_columns(
    (((pl.col("target_amount") / pl.col("price") * 1000).floor()) / 1000.0).alias("target_qty")
)

test_df = test_df.with_columns(
    (
        pl.col("target_qty") * pl.col("price")
    ).alias("rounded_amount")
)




test_df = test_df.with_columns(
    (
        pl.col("target_qty") - pl.col("target_qty").shift(1).fill_null(0)
    ).alias("delta_qty")
)



test_df = test_df.with_columns(
    (
        pl.col("target_qty").shift(1).fill_null(0)
    ).alias("cum_qty")
)






test_df = test_df.with_columns(
    (
        pl.col("delta_qty").abs()*pl.col("price")*0.06/100
    ).alias("trans_cost")
)


test_df = test_df.with_columns(
    (
        # (pl.col("target_qty").shift(1) * pl.col("price").shift(1)) * pl.col("price_chg")
        (
            pl.col("target_qty").shift(1) * 
            (pl.col("price")- pl.col("price").shift(1) )
        ).fill_null(0)
        - pl.col("trans_cost")
    ).alias("PnL(test)")
)

test_df = test_df.with_columns(
    (
        pl.col("target_qty").shift(1).fill_null(0) * pl.col("price") 
        # - pl.col("position").shift(1).fill_null(0) * betsize 
        -pl.col("rounded_amount").shift(1).fill_null(0)
        - pl.col("trans_cost")
    ).alias("PnL")
)

test_df = test_df.with_columns(
    (
        pl.col("PnL").cum_sum()
    ).alias("cum_PnL")
)

test_df = test_df.with_columns(
    (
        pl.col("cum_PnL") + betsize
    ).alias("cum_holding")
)

In [176]:
test_df

timestamp,position,price,price_chg,target_amount,target_qty,rounded_amount,delta_qty,cum_qty,trans_cost,PnL(test),PnL,cum_PnL,cum_holding
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,0.5,60000.0,0.0,5000.0,0.083,4980.0,0.083,0.0,2.988,-2.988,-2.988,-2.988,9997.012
2,1.0,65000.0,0.083333,10000.0,0.153,9945.0,0.07,0.083,2.73,412.27,412.27,409.282,10409.282
3,1.0,70000.0,0.076923,10000.0,0.142,9940.0,-0.011,0.153,0.462,764.538,764.538,1173.82,11173.82
4,0.0,71000.0,0.014286,0.0,0.0,0.0,-0.142,0.142,6.0492,135.9508,135.9508,1309.7708,11309.7708
5,0.0,80000.0,0.126761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1309.7708,11309.7708


In [177]:
0.153 * (70000 - 65000) - 0.142*(5000)

55.000000000000114

In [178]:
0.153 * (70000 - 65000) 

765.0

In [179]:
 0.011 * (5000) * ( 1- 0.06/100)

54.967